<a href="https://colab.research.google.com/github/mounsifelatouch/cdd/blob/master/notebooks/4_cdd_ml_part_4_model_building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bioinformatics Project - Computational Drug Discovery [Part 4] Classification Models Building**

**MOUNSIF EL ATOUCH**

In this Jupyter notebook, we will be building a machine learning model using the ChEMBL bioactivity data.

In **Part 4**, we will be building classification models

---

## **1. Installing librairies**

## **2. Importing libraries**

In [387]:
import os
import pandas as pd

from scipy.spatial.distance import *
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

seed = 123

from sklearn.model_selection import *
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import *

from scipy.stats import randint, uniform

from scipy import interp
from sklearn.metrics import RocCurveDisplay

import warnings
warnings.filterwarnings('ignore')

In [388]:
from sklearn.ensemble import ExtraTreesClassifier
import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
import lightgbm as lgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.svm import SVC  
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

In [389]:
def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n")
        
    elif train==False:
        pred = clf.predict(X_test)
        clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
        print("Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")

In [390]:
def print_auc(clf, X_train, y_train, X_test, y_test, model_name) :
    
    clf = clf.fit(X_train, y_train)
    y_pred_proba = clf.predict_proba(X_test)[::,1]
    fpr, tpr, thr = roc_curve(y_test,  y_pred_proba)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    #plt.plot(fpr, tpr, label=model_name+" auc="+str(roc_auc))
    #plt.legend(loc=4)
    #plt.title("ROC curve")

    #plt.show()
    # Data to plot precision - recall curve
    precision, recall, thresholds_log = precision_recall_curve(y_test, y_pred_proba)
    # Use AUC function to calculate the area under the curve of precision recall curve
    auc_precision_recall = auc(recall, precision)
    
    #plt.plot(recall, precision, label=model_name+" auc_prc="+str(auc_precision_recall))
    #plt.xlabel("Recall")
    #plt.ylabel("Precision")
    #plt.legend(loc=4)
    #plt.title("Precision-Recall Curve")

    #plt.show()
    precision_sc=precision_score(y_test, clf.predict(X_test), average=None)
    recall_sc = recall_score(y_test, clf.predict(X_test), average=None)
    test_score = accuracy_score(y_test, clf.predict(X_test)) * 100
    train_score = accuracy_score(y_train, clf.predict(X_train)) * 100
    f1 = f1_score(y_test, clf.predict(X_test))
    #print_score(clf, X_train, y_train, X_test, y_test, train=True)
    #print_score(clf, X_train, y_train, X_test, y_test, train=False)
    metriques = {"fpr" : fpr, "recall_sc" : recall_sc, "precision_sc" : precision_sc, "tpr" : tpr,
                 "test_score" : test_score, "train_score" : train_score, "f1" : f1,
                 "auc_pr" : auc_precision_recall, "auc": roc_auc}
    return metriques

In [391]:
def metrics(X_train, X_test, y_train, y_test, model):
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    print("Training set accuracy: {:.2f}".format(model.score(X_train, y_train)))
    print("Test set accuracy: {:.2f}".format(model.score(X_test, y_test)))
    print("Precision: {:.2f}".format(precision_score(y_test, y_pred)))
    print("Recall: {:.2f}".format(recall_score(y_test, y_pred)))
    print("F1 score: {:.2f}".format(f1_score(y_test, y_pred)))
    print("ROC AUC score: {:.2f}".format(roc_auc_score(y_test, y_proba)))
    print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    print("True Negatives: {:.0f}".format(tn))
    print("False Positives: {:.0f}".format(fp))
    print("False Negatives: {:.0f}".format(fn))
    print("TruePositives: {:.0f}".format(tp))
    print("-------------------------------------")
    return

## **4. Load the data set**

In [392]:
! wget https://raw.githubusercontent.com/mounsifelatouch/cdd/master/data/bioactivity_data_PubchemFingerprinter.csv

--2023-06-06 13:07:34--  https://raw.githubusercontent.com/mounsifelatouch/cdd/master/data/bioactivity_data_PubchemFingerprinter.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1090920 (1.0M) [text/plain]
Saving to: 'bioactivity_data_PubchemFingerprinter.csv.3'

     0K .......... .......... .......... .......... ..........  4%  519K 2s
    50K .......... .......... .......... .......... ..........  9%  754K 2s
   100K .......... .......... .......... .......... .......... 14%  731K 1s
   150K .......... .......... .......... .......... .......... 18%  749K 1s
   200K .......... .......... .......... .......... .......... 23%  707K 1s
   250K .......... .......... .......... .......... .......... 28%  774K 1s
   300K .......... .......... .......... ....

In [393]:
df = pd.read_csv('bioactivity_data_PubchemFingerprinter.csv')

## **5. Input features**

### **5.1. Input features**

In [394]:
X = df.drop('activity', axis=1)

### **5.2. Output features**

In [395]:
y = df['activity']

In [396]:
# count the number of instances in each class
counts = y.value_counts()

# calculate the ratio of the negative class to the positive class
imbalance_ratio = counts[0] / counts[1]

### **5.3. Let's examine the data dimension**

In [397]:
X.shape, y.shape

((612, 881), (612,))

### **5.4. Remove low variance features**

In [398]:
# Create a VarianceThreshold object
selector = VarianceThreshold(threshold=.01)

# Fit the selector to the data and transform the data
selector.fit_transform(X)

# Get the indices of the selected features
selected_indices = selector.get_support(indices=True)

# Get the names of the selected features
selected_features = X.columns[selected_indices]

X_reduced = X[selected_features]

In [399]:
X.shape, X_reduced.shape

((612, 881), (612, 428))

## **6. Data split (80/20 ratio)**

In [400]:
n = np.arange(len(X_reduced))
idx_train, idx_test = train_test_split(n, stratify=y, test_size=.2, random_state=seed)

In [401]:
X1_train, y1_train = X_reduced.loc[idx_train], y.loc[idx_train]
X1_test, y1_test = X_reduced.loc[idx_test], y.loc[idx_test]

In [402]:
X1_train.shape, y1_train.shape

((489, 428), (489,))

In [403]:
X1_test.shape, y1_test.shape

((123, 428), (123,))



---



### **Random Oversampling**

In [404]:
from imblearn.over_sampling import RandomOverSampler

# create an oversampler object with a 1:1 ratio of positive to negative samples
ros = RandomOverSampler(sampling_strategy='minority', random_state=seed)

# fit and apply the oversampler to the training data
X1_train_ros, y1_train_ros = ros.fit_resample(X1_train, y1_train)

In [405]:
X = X1_train_ros.reset_index(drop=True)
y = y1_train_ros.reset_index(drop=True)
X1_test = X1_test.reset_index(drop=True)
y1_test = y1_test.reset_index(drop=True)

In [406]:
n_samples, n_classes = X.shape[0], 2
class_weights = dict(zip(np.unique(y), n_samples / (n_classes * np.bincount(y))))

## **7. Modeling**

## **Logistic Regression**

from sklearn.linear_model import LogisticRegression

clf1 = LogisticRegression()

cross_val = StratifiedKFold(n_splits=5)

fig, ax = plt.subplots(figsize=(5.5, 5.5))

tprs, aucs, y_real, y_prob, prs = [], [], [], [], []

mean_recall = np.linspace(0, 1, 100)
    
plt.figure(figsize=(5.5 , 5.5))

accs, precs, recs, f1_scr = [], [], [], []

mean_recall = np.linspace(0, 1, 100)
mean_fpr = np.linspace(0, 1, 100)

i=0

space = dict()
space['class_weight'] = [class_weights, None, 'balanced']
space['solver'] = ['lbfgs', 'liblinear', 'newton-cg', 'sag']
space['penalty'] = ['l1', 'l2', 'none']
space['C'] = uniform(0.1, 100)
space['max_iter'] = randint(100, 500)


train_accuracy_list = []
test_accuracy_list = []
lst_accu_stratified = []

index_iterator = cross_val.split(X, y)

clf_random = RandomizedSearchCV(estimator=clf1, param_distributions=space, n_iter=100, scoring='f1', cv=cross_val, verbose=2, n_jobs=-1, random_state=seed)
for train_index, test_index in index_iterator:
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
  
    clf_random.fit(X_train, y_train)
    pred_proba = clf_random.predict_proba(X_test)
    
    precision, recall, _ = precision_recall_curve(y_test, pred_proba[:, 1])
    prs.append(interp(mean_recall, precision, recall))
    pr_auc = auc(recall, precision)
    aucs.append(pr_auc)
    plt.plot(recall, precision, lw=3, alpha=0.5, label='Fold %d (AUCPR = %0.2f)' % (i+1, pr_auc))

    viz = RocCurveDisplay.from_estimator(clf_random, X_test, y_test, name='ROC fold {}'.format(i+1), ax=ax, alpha=.3)

    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)

    lst_accu_stratified.append(clf_random.score(X_test, y_test))
    y_train_pred = clf_random.predict(X_train)
    train_accuracy = np.mean(y_train_pred.ravel() == y_train.ravel())*100
    train_accuracy_list.append(train_accuracy)
    y_test_pred = clf_random.predict(X_test)
    test_accuracy = np.mean(y_test_pred.ravel() == y_test.ravel())*100
    test_accuracy_list.append(test_accuracy)
    print('--------------------------------------------------------------------')
    print('Best Score: %s'.format(i) % clf_random.best_score_)
    print('Best Hyperparameters: %s'.format(i) % clf_random.best_params_)
    acc = accuracy_score(y_test, y_test_pred)
    prec = precision_score(y_test, y_test_pred)
    rec = recall_score(y_test, y_test_pred)
    f1 = f1_score(y_test,y_test_pred)
    accs.append(acc)
    precs.append(prec)
    recs.append(rec)
    f1_scr.append(f1)
    print(f'Accuracy: {acc}, Precision: {prec}, Recall: {rec}, f1_score: {f1}')
    i=i+1


ax.plot([0, 1], [0, 1], linestyle='--', lw=.7, color='black', label='Chance', alpha=.7)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color='blue', label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc), lw=.7, alpha=.7)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.7, label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05], title="Receiver operating characteristic for LGBM")
ax.legend(loc="lower right")
plt.show()

plt.plot([0, 1], [1, 0], linestyle='--', lw=.7, color='k', label='Luck', alpha=.7)
mean_precision = np.mean(prs, axis=0)
mean_auc = auc(mean_recall, mean_precision)
std_auc = np.std(aucs)
plt.plot(mean_precision, mean_recall, color='navy',label=r'Mean (AUCPR = %0.3f $\pm$ %0.2f)' % (mean_auc, std_auc),lw=4)
    
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('Recall' ,  fontweight = "bold" , fontsize=10)
plt.ylabel('Precision',fontweight = "bold" , fontsize=10)
plt.tick_params(axis='both', which='major', labelsize=10)
plt.legend( prop={'size':10} , loc = 0)
plt.show()

from statistics import mean
from statistics import stdev

print('List of possible accuracy: \n', lst_accu_stratified)
print('Maximum Accuracy That can be obtained from this model is: \n', max(lst_accu_stratified)*100, '%')
print('Minimum Accuracy: \n', min(lst_accu_stratified)*100, '%')
print('Overall Accuracy: \n', mean(lst_accu_stratified)*100, '%')
print('Standard Deviation is: \n', stdev(lst_accu_stratified))

print('List of possible accuracy for trainning:', train_accuracy_list)
print('List of possible accuracy for testing:', test_accuracy_list)

print(f'Mean Accuracy: {np.mean(accs)}, Mean Precision: {np.mean(precs)}, Mean Recall: {np.mean(recs)}, Mean f1_score: {np.mean(f1)}')

In [407]:
model1 = LogisticRegression(random_state=seed).fit(X1_train, y1_train)

In [408]:
clf = print_auc(model1, X1_train, y1_train, X1_test, y1_test, 'Logistic Regression')
results_df = pd.DataFrame(data=[['Logistic Regression', clf['f1'], clf['auc'], clf['auc_pr'], clf['recall_sc'][0], clf['recall_sc'][1], clf['precision_sc'][0], clf['precision_sc'][1]]], 
                          columns=['Model', 'f1_score', 'auc', 'auc_pr', 'recall_classe(0)', 'recall_classe(1)', 'precision_classe(0)', 'precision_classe(1)'])

results_df.style.hide_index()

Model,f1_score,auc,auc_pr,recall_classe(0),recall_classe(1),precision_classe(0),precision_classe(1)
Logistic Regression,0.888889,0.953297,0.932885,0.967033,0.875000,0.956522,0.903226


## **SVC**

from sklearn.svm import SVC  

clf1 = SVC(probability=True)

cross_val = StratifiedKFold(n_splits=5)

fig, ax = plt.subplots(figsize=(5.5, 5.5))

tprs, aucs, y_real, y_prob, prs = [], [], [], [], []

mean_recall = np.linspace(0, 1, 100)
    
plt.figure(figsize=(5.5 , 5.5))

accs, precs, recs, f1_scr = [], [], [], []

mean_recall = np.linspace(0, 1, 100)
mean_fpr = np.linspace(0, 1, 100)

i=0

param_grid = {'C': uniform(0.1, 100),
              'degree': randint(2, 5),
              'gamma': ['scale', 'auto'],
              'class_weight': [None, 'balanced', class_weights]}

train_accuracy_list = []
test_accuracy_list = []
lst_accu_stratified = []

index_iterator = cross_val.split(X, y)

clf_random = clf_random = RandomizedSearchCV(estimator=clf1, param_distributions=param_grid, n_iter=100, scoring='f1', cv=cross_val, verbose=2, n_jobs=-1, random_state=seed)
for train_index, test_index in index_iterator:
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
  
    clf_random.fit(X_train, y_train)
    pred_proba = clf_random.predict_proba(X_test)
    
    precision, recall, _ = precision_recall_curve(y_test, pred_proba[:, 1])
    prs.append(interp(mean_recall, precision, recall))
    pr_auc = auc(recall, precision)
    aucs.append(pr_auc)
    plt.plot(recall, precision, lw=3, alpha=0.5, label='Fold %d (AUCPR = %0.2f)' % (i+1, pr_auc))

    viz = RocCurveDisplay.from_estimator(clf_random, X_test, y_test, name='ROC fold {}'.format(i+1), ax=ax, alpha=.3)

    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)

    lst_accu_stratified.append(clf_random.score(X_test, y_test))
    y_train_pred = clf_random.predict(X_train)
    train_accuracy = np.mean(y_train_pred.ravel() == y_train.ravel())*100
    train_accuracy_list.append(train_accuracy)
    y_test_pred = clf_random.predict(X_test)
    test_accuracy = np.mean(y_test_pred.ravel() == y_test.ravel())*100
    test_accuracy_list.append(test_accuracy)
    print('--------------------------------------------------------------------')
    print('Best Score: %s'.format(i) % clf_random.best_score_)
    print('Best Hyperparameters: %s'.format(i) % clf_random.best_params_)
    acc = accuracy_score(y_test, y_test_pred)
    prec = precision_score(y_test, y_test_pred)
    rec = recall_score(y_test, y_test_pred)
    f1 = f1_score(y_test,y_test_pred)
    accs.append(acc)
    precs.append(prec)
    recs.append(rec)
    f1_scr.append(f1)
    print(f'Accuracy: {acc}, Precision: {prec}, Recall: {rec}, f1_score: {f1}')
    i=i+1


ax.plot([0, 1], [0, 1], linestyle='--', lw=.7, color='black', label='Chance', alpha=.7)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color='blue', label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc), lw=.7, alpha=.7)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.7, label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05], title="Receiver operating characteristic for LGBM")
ax.legend(loc="lower right")
plt.show()

plt.plot([0, 1], [1, 0], linestyle='--', lw=.7, color='k', label='Luck', alpha=.7)
mean_precision = np.mean(prs, axis=0)
mean_auc = auc(mean_recall, mean_precision)
std_auc = np.std(aucs)
plt.plot(mean_precision, mean_recall, color='navy',label=r'Mean (AUCPR = %0.3f $\pm$ %0.2f)' % (mean_auc, std_auc),lw=4)
    
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('Recall' ,  fontweight = "bold" , fontsize=10)
plt.ylabel('Precision',fontweight = "bold" , fontsize=10)
plt.tick_params(axis='both', which='major', labelsize=10)
plt.legend( prop={'size':10} , loc = 0)
plt.show()

from statistics import mean
from statistics import stdev

print('List of possible accuracy: \n', lst_accu_stratified)
print('Maximum Accuracy That can be obtained from this model is: \n', max(lst_accu_stratified)*100, '%')
print('Minimum Accuracy: \n', min(lst_accu_stratified)*100, '%')
print('Overall Accuracy: \n', mean(lst_accu_stratified)*100, '%')
print('Standard Deviation is: \n', stdev(lst_accu_stratified))

print('List of possible accuracy for trainning:', train_accuracy_list)
print('List of possible accuracy for testing:', test_accuracy_list)

print(f'Mean Accuracy: {np.mean(accs)}, Mean Precision: {np.mean(precs)}, Mean Recall: {np.mean(recs)}, Mean f1_score: {np.mean(f1)}')

In [409]:
model2 = SVC(random_state=seed, probability=True).fit(X1_train, y1_train)

In [410]:
clf = print_auc(model2, X1_train, y1_train, X1_test, y1_test, 'SVM')
results_df2 = pd.DataFrame(data=[['SVM', clf['f1'], clf['auc'], clf['auc_pr'], clf['recall_sc'][0], clf['recall_sc'][1], clf['precision_sc'][0], clf['precision_sc'][1]]],  
                          columns=['Model', 'f1_score', 'auc','auc_pr','recall_classe(0)','recall_classe(1)','precision_classe(0)','precision_classe(1)'])
results_df = results_df.append(results_df2)

results_df.style.hide_index()

Model,f1_score,auc,auc_pr,recall_classe(0),recall_classe(1),precision_classe(0),precision_classe(1)
Logistic Regression,0.888889,0.953297,0.932885,0.967033,0.875000,0.956522,0.903226
SVM,0.847458,0.966346,0.927824,0.978022,0.781250,0.927083,0.925926


## **Gradient Boosting Classifier**

from sklearn.ensemble import GradientBoostingClassifier

clf1 = GradientBoostingClassifier()

cross_val = StratifiedKFold(n_splits=5)

fig, ax = plt.subplots(figsize=(5.5, 5.5))

tprs, aucs, y_real, y_prob, prs = [], [], [], [], []

mean_recall = np.linspace(0, 1, 100)
    
plt.figure(figsize=(5.5 , 5.5))

accs, precs, recs, f1_scr = [], [], [], []

mean_recall = np.linspace(0, 1, 100)
mean_fpr = np.linspace(0, 1, 100)

i=0

param_grid = {'learning_rate': uniform(1e-5, 1),
              'subsample': uniform(0.8, 1),
              'n_estimators' : randint(100, 1000),
              'max_features' : ['log2', 'sqrt', 'auto'],
              'max_depth': randint(3, 10)}

train_accuracy_list = []
test_accuracy_list = []
lst_accu_stratified = []

index_iterator = cross_val.split(X, y)

clf_random = clf_random = RandomizedSearchCV(estimator=clf1, param_distributions=param_grid, n_iter=100, scoring='f1', cv=cross_val, verbose=2, n_jobs=-1, random_state=seed)
for train_index, test_index in index_iterator:
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
  
    clf_random.fit(X_train, y_train)
    pred_proba = clf_random.predict_proba(X_test)
    
    precision, recall, _ = precision_recall_curve(y_test, pred_proba[:, 1])
    prs.append(interp(mean_recall, precision, recall))
    pr_auc = auc(recall, precision)
    aucs.append(pr_auc)
    plt.plot(recall, precision, lw=3, alpha=0.5, label='Fold %d (AUCPR = %0.2f)' % (i+1, pr_auc))

    viz = RocCurveDisplay.from_estimator(clf_random, X_test, y_test, name='ROC fold {}'.format(i+1), ax=ax, alpha=.3)

    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)

    lst_accu_stratified.append(clf_random.score(X_test, y_test))
    y_train_pred = clf_random.predict(X_train)
    train_accuracy = np.mean(y_train_pred.ravel() == y_train.ravel())*100
    train_accuracy_list.append(train_accuracy)
    y_test_pred = clf_random.predict(X_test)
    test_accuracy = np.mean(y_test_pred.ravel() == y_test.ravel())*100
    test_accuracy_list.append(test_accuracy)
    print('--------------------------------------------------------------------')
    print('Best Score: %s'.format(i) % clf_random.best_score_)
    print('Best Hyperparameters: %s'.format(i) % clf_random.best_params_)
    acc = accuracy_score(y_test, y_test_pred)
    prec = precision_score(y_test, y_test_pred)
    rec = recall_score(y_test, y_test_pred)
    f1 = f1_score(y_test,y_test_pred)
    accs.append(acc)
    precs.append(prec)
    recs.append(rec)
    f1_scr.append(f1)
    print(f'Accuracy: {acc}, Precision: {prec}, Recall: {rec}, f1_score: {f1}')
    i=i+1


ax.plot([0, 1], [0, 1], linestyle='--', lw=.7, color='black', label='Chance', alpha=.7)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color='blue', label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc), lw=.7, alpha=.7)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.7, label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05], title="Receiver operating characteristic for LGBM")
ax.legend(loc="lower right")
plt.show()

plt.plot([0, 1], [1, 0], linestyle='--', lw=.7, color='k', label='Luck', alpha=.7)
mean_precision = np.mean(prs, axis=0)
mean_auc = auc(mean_recall, mean_precision)
std_auc = np.std(aucs)
plt.plot(mean_precision, mean_recall, color='navy',label=r'Mean (AUCPR = %0.3f $\pm$ %0.2f)' % (mean_auc, std_auc),lw=4)
    
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('Recall' ,  fontweight = "bold" , fontsize=10)
plt.ylabel('Precision',fontweight = "bold" , fontsize=10)
plt.tick_params(axis='both', which='major', labelsize=10)
plt.legend( prop={'size':10} , loc = 0)
plt.show()

from statistics import mean
from statistics import stdev

print('List of possible accuracy: \n', lst_accu_stratified)
print('Maximum Accuracy That can be obtained from this model is: \n', max(lst_accu_stratified)*100, '%')
print('Minimum Accuracy: \n', min(lst_accu_stratified)*100, '%')
print('Overall Accuracy: \n', mean(lst_accu_stratified)*100, '%')
print('Standard Deviation is: \n', stdev(lst_accu_stratified))

print('List of possible accuracy for trainning:', train_accuracy_list)
print('List of possible accuracy for testing:', test_accuracy_list)

print(f'Mean Accuracy: {np.mean(accs)}, Mean Precision: {np.mean(precs)}, Mean Recall: {np.mean(recs)}, Mean f1_score: {np.mean(f1)}')

In [411]:
model3 = GradientBoostingClassifier(random_state=seed).fit(X1_train, y1_train)

In [412]:
clf = print_auc(model3, X1_train, y1_train, X1_test, y1_test, 'Gradient Boosting')
results_df3 = pd.DataFrame(data=[['Gradient Boosting', clf['f1'], clf['auc'], clf['auc_pr'], clf['recall_sc'][0], clf['recall_sc'][1], clf['precision_sc'][0], clf['precision_sc'][1]]], 
                          columns=['Model', 'f1_score', 'auc','auc_pr','recall_classe(0)','recall_classe(1)','precision_classe(0)','precision_classe(1)'])
results_df = results_df.append(results_df3)

results_df.style.hide_index()

Model,f1_score,auc,auc_pr,recall_classe(0),recall_classe(1),precision_classe(0),precision_classe(1)
Logistic Regression,0.888889,0.953297,0.932885,0.967033,0.875000,0.956522,0.903226
SVM,0.847458,0.966346,0.927824,0.978022,0.781250,0.927083,0.925926
Gradient Boosting,0.875000,0.959135,0.936712,0.956044,0.875000,0.956044,0.875000


## **Random Forest**

from sklearn.ensemble import RandomForestClassifier

clf1 = RandomForestClassifier(random_state=seed)

cross_val = StratifiedKFold(n_splits=5)

fig, ax = plt.subplots(figsize=(5.5, 5.5))

tprs, aucs, y_real, y_prob, prs = [], [], [], [], []

mean_recall = np.linspace(0, 1, 100)
    
plt.figure(figsize=(5.5 , 5.5))

accs, precs, recs, f1_scr = [], [], [], []

mean_recall = np.linspace(0, 1, 100)
mean_fpr = np.linspace(0, 1, 100)

i=0

param_grid = {'n_estimators': randint(50, 1000),
              'max_depth': randint(3, 10),
              'min_samples_split': randint(2, 10),
              'min_samples_leaf': randint(1, 5),
              'max_features': ['sqrt', 'log2', None],
              'criterion': ['gini', 'entropy', 'log_loss'],
              'class_weight' : ['balanced', None, class_weights]}

train_accuracy_list = []
test_accuracy_list = []
lst_accu_stratified = []

index_iterator = cross_val.split(X, y)

clf_random = clf_random = RandomizedSearchCV(estimator=clf1, param_distributions=param_grid, n_iter=100, scoring='f1', cv=cross_val, verbose=2, n_jobs=-1, random_state=seed)
for train_index, test_index in index_iterator:
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
  
    clf_random.fit(X_train, y_train)
    pred_proba = clf_random.predict_proba(X_test)
    
    precision, recall, _ = precision_recall_curve(y_test, pred_proba[:, 1])
    prs.append(interp(mean_recall, precision, recall))
    pr_auc = auc(recall, precision)
    aucs.append(pr_auc)
    plt.plot(recall, precision, lw=3, alpha=0.5, label='Fold %d (AUCPR = %0.2f)' % (i+1, pr_auc))

    viz = RocCurveDisplay.from_estimator(clf_random, X_test, y_test, name='ROC fold {}'.format(i+1), ax=ax, alpha=.3)

    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)

    lst_accu_stratified.append(clf_random.score(X_test, y_test))
    y_train_pred = clf_random.predict(X_train)
    train_accuracy = np.mean(y_train_pred.ravel() == y_train.ravel())*100
    train_accuracy_list.append(train_accuracy)
    y_test_pred = clf_random.predict(X_test)
    test_accuracy = np.mean(y_test_pred.ravel() == y_test.ravel())*100
    test_accuracy_list.append(test_accuracy)
    print('--------------------------------------------------------------------')
    print('Best Score: %s'.format(i) % clf_random.best_score_)
    print('Best Hyperparameters: %s'.format(i) % clf_random.best_params_)
    acc = accuracy_score(y_test, y_test_pred)
    prec = precision_score(y_test, y_test_pred)
    rec = recall_score(y_test, y_test_pred)
    f1 = f1_score(y_test,y_test_pred)
    accs.append(acc)
    precs.append(prec)
    recs.append(rec)
    f1_scr.append(f1)
    print(f'Accuracy: {acc}, Precision: {prec}, Recall: {rec}, f1_score: {f1}')
    i=i+1


ax.plot([0, 1], [0, 1], linestyle='--', lw=.7, color='black', label='Chance', alpha=.7)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color='blue', label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc), lw=.7, alpha=.7)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.7, label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05], title="Receiver operating characteristic for LGBM")
ax.legend(loc="lower right")
plt.show()

plt.plot([0, 1], [1, 0], linestyle='--', lw=.7, color='k', label='Luck', alpha=.7)
mean_precision = np.mean(prs, axis=0)
mean_auc = auc(mean_recall, mean_precision)
std_auc = np.std(aucs)
plt.plot(mean_precision, mean_recall, color='navy',label=r'Mean (AUCPR = %0.3f $\pm$ %0.2f)' % (mean_auc, std_auc),lw=4)
    
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('Recall' ,  fontweight = "bold" , fontsize=10)
plt.ylabel('Precision',fontweight = "bold" , fontsize=10)
plt.tick_params(axis='both', which='major', labelsize=10)
plt.legend( prop={'size':10} , loc = 0)
plt.show()

from statistics import mean
from statistics import stdev

print('List of possible accuracy:', lst_accu_stratified)
print('\nMaximum Accuracy That can be obtained from this model is:', max(lst_accu_stratified)*100, '%')
print('\nMinimum Accuracy:', min(lst_accu_stratified)*100, '%')
print('\nOverall Accuracy:', mean(lst_accu_stratified)*100, '%')
print('\nStandard Deviation is:', stdev(lst_accu_stratified))

print('List of possible accuracy for trainning:', train_accuracy_list)
print('List of possible accuracy for testing:', test_accuracy_list)

print(f'Mean Accuracy: {np.mean(accs)}, Mean Precision: {np.mean(precs)}, Mean Recall: {np.mean(recs)}, Mean f1_score: {np.mean(f1)}')

In [413]:
model4 = RandomForestClassifier(random_state=seed).fit(X1_train, y1_train)

In [414]:
clf = print_auc(model4, X1_train, y1_train, X1_test, y1_test, 'Random Forest')
results_df3 = pd.DataFrame(data=[['Random Forest', clf['f1'], clf['auc'], clf['auc_pr'], clf['recall_sc'][0], clf['recall_sc'][1], clf['precision_sc'][0], clf['precision_sc'][1]]], 
                          columns=['Model', 'f1_score', 'auc', 'auc_pr', 'recall_classe(0)', 'recall_classe(1)', 'precision_classe(0)', 'precision_classe(1)'])
results_df = results_df.append(results_df3, ignore_index=True)

results_df.style.hide_index()

Model,f1_score,auc,auc_pr,recall_classe(0),recall_classe(1),precision_classe(0),precision_classe(1)
Logistic Regression,0.888889,0.953297,0.932885,0.967033,0.875000,0.956522,0.903226
SVM,0.847458,0.966346,0.927824,0.978022,0.781250,0.927083,0.925926
Gradient Boosting,0.875000,0.959135,0.936712,0.956044,0.875000,0.956044,0.875000
Random Forest,0.852459,0.965659,0.927233,0.967033,0.812500,0.936170,0.896552


## **KNN**

from sklearn.neighbors import KNeighborsClassifier

clf1 = KNeighborsClassifier()

cross_val = StratifiedKFold(n_splits=5)

fig, ax = plt.subplots(figsize=(5.5, 5.5))

tprs, aucs, y_real, y_prob, prs = [], [], [], [], []

mean_recall = np.linspace(0, 1, 100)
    
plt.figure(figsize=(5.5 , 5.5))

accs, precs, recs, f1_scr = [], [], [], []

mean_recall = np.linspace(0, 1, 100)
mean_fpr = np.linspace(0, 1, 100)

i=0

param_grid = {'n_neighbors': randint(3, 7),
              'weights': ['uniform', 'distance'],
              'p': [1, 2],
              'algorithm': ['brute'],
              'leaf_size': randint(10, 50)}

train_accuracy_list = []
test_accuracy_list = []
lst_accu_stratified = []

index_iterator = cross_val.split(X, y)

clf_random = clf_random = RandomizedSearchCV(estimator=clf1, param_distributions=param_grid, n_iter=100, scoring='f1', cv=cross_val, verbose=2, n_jobs=-1, random_state=seed)

for train_index, test_index in index_iterator:
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
  
    clf_random.fit(X_train, y_train)
    pred_proba = clf_random.predict_proba(X_test)
    
    precision, recall, _ = precision_recall_curve(y_test, pred_proba[:, 1])
    prs.append(interp(mean_recall, precision, recall))
    pr_auc = auc(recall, precision)
    aucs.append(pr_auc)
    plt.plot(recall, precision, lw=3, alpha=0.5, label='Fold %d (AUCPR = %0.2f)' % (i+1, pr_auc))

    viz = RocCurveDisplay.from_estimator(clf_random, X_test, y_test, name='ROC fold {}'.format(i+1), ax=ax, alpha=.3)

    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)

    lst_accu_stratified.append(clf_random.score(X_test, y_test))
    y_train_pred = clf_random.predict(X_train)
    train_accuracy = np.mean(y_train_pred.ravel() == y_train.ravel())*100
    train_accuracy_list.append(train_accuracy)
    y_test_pred = clf_random.predict(X_test)
    test_accuracy = np.mean(y_test_pred.ravel() == y_test.ravel())*100
    test_accuracy_list.append(test_accuracy)
    print('--------------------------------------------------------------------')
    print('Best Score: %s'.format(i) % clf_random.best_score_)
    print('Best Hyperparameters: %s'.format(i) % clf_random.best_params_)
    acc = accuracy_score(y_test, y_test_pred)
    prec = precision_score(y_test, y_test_pred)
    rec = recall_score(y_test, y_test_pred)
    f1 = f1_score(y_test,y_test_pred)
    accs.append(acc)
    precs.append(prec)
    recs.append(rec)
    f1_scr.append(f1)
    print(f'Accuracy: {acc}, Precision: {prec}, Recall: {rec}, f1_score: {f1}')
    i=i+1


ax.plot([0, 1], [0, 1], linestyle='--', lw=.7, color='black', label='Chance', alpha=.7)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color='blue', label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc), lw=.7, alpha=.7)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.7, label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05], title="Receiver operating characteristic for LGBM")
ax.legend(loc="lower right")
plt.show()

plt.plot([0, 1], [1, 0], linestyle='--', lw=.7, color='k', label='Luck', alpha=.7)
mean_precision = np.mean(prs, axis=0)
mean_auc = auc(mean_recall, mean_precision)
std_auc = np.std(aucs)
plt.plot(mean_precision, mean_recall, color='navy',label=r'Mean (AUCPR = %0.3f $\pm$ %0.2f)' % (mean_auc, std_auc),lw=4)
    
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('Recall' ,  fontweight = "bold" , fontsize=10)
plt.ylabel('Precision',fontweight = "bold" , fontsize=10)
plt.tick_params(axis='both', which='major', labelsize=10)
plt.legend( prop={'size':10} , loc = 0)
plt.show()

from statistics import mean
from statistics import stdev

print('List of possible accuracy: \n', lst_accu_stratified)
print('Maximum Accuracy That can be obtained from this model is: \n', max(lst_accu_stratified)*100, '%')
print('Minimum Accuracy: \n', min(lst_accu_stratified)*100, '%')
print('Overall Accuracy: \n', mean(lst_accu_stratified)*100, '%')
print('Standard Deviation is: \n', stdev(lst_accu_stratified))

print('List of possible accuracy for trainning:', train_accuracy_list)
print('List of possible accuracy for testing:', test_accuracy_list)

print(f'Mean Accuracy: {np.mean(accs)}, Mean Precision: {np.mean(precs)}, Mean Recall: {np.mean(recs)}, Mean f1_score: {np.mean(f1)}')

In [415]:
model5 = KNeighborsClassifier().fit(X1_train, y1_train)

In [416]:
clf = print_auc(model5, X1_train, y1_train, X1_test, y1_test, 'KNeibors Classifier')
results_df4 = pd.DataFrame(data=[['KNeibors Classifier', clf['f1'], clf['auc'], clf['auc_pr'], clf['recall_sc'][0], clf['recall_sc'][1], clf['precision_sc'][0], clf['precision_sc'][1]]], 
                          columns=['Model', 'f1_score', 'auc', 'auc_pr', 'recall_classe(0)', 'recall_classe(1)', 'precision_classe(0)', 'precision_classe(1)'])
results_df = results_df.append(results_df4, ignore_index=True)

results_df.style.hide_index()

Model,f1_score,auc,auc_pr,recall_classe(0),recall_classe(1),precision_classe(0),precision_classe(1)
Logistic Regression,0.888889,0.953297,0.932885,0.967033,0.875000,0.956522,0.903226
SVM,0.847458,0.966346,0.927824,0.978022,0.781250,0.927083,0.925926
Gradient Boosting,0.875000,0.959135,0.936712,0.956044,0.875000,0.956044,0.875000
Random Forest,0.852459,0.965659,0.927233,0.967033,0.812500,0.936170,0.896552
KNeibors Classifier,0.833333,0.955357,0.917124,0.967033,0.781250,0.926316,0.892857


## **ExtraTrees**

from sklearn.ensemble import ExtraTreesClassifier

clf1 = ExtraTreesClassifier(random_state=0)

cross_val = StratifiedKFold(n_splits=5)

fig, ax = plt.subplots(figsize=(5.5, 5.5))

tprs, aucs, y_real, y_prob, prs = [], [], [], [], []

mean_recall = np.linspace(0, 1, 100)
    
plt.figure(figsize=(5.5 , 5.5))

accs, precs, recs, f1_scr = [], [], [], []

mean_recall = np.linspace(0, 1, 100)
mean_fpr = np.linspace(0, 1, 100)

i=0

param_grid = {'n_estimators': randint(100, 1000),
              'max_depth': randint(10, 100),
              'max_features': ['sqrt', 'log2'],
              'min_samples_split' : randint(2, 10), 
              'min_samples_leaf' : randint(1, 10),
              'class_weight' : ['balanced', None, class_weights]}

train_accuracy_list = []
test_accuracy_list = []
lst_accu_stratified = []

index_iterator = cross_val.split(X, y)

clf_random = clf_random = RandomizedSearchCV(estimator=clf1, param_distributions=param_grid, n_iter=100, scoring='f1', cv=cross_val, verbose=2, n_jobs=-1, random_state=seed)
for train_index, test_index in index_iterator:
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
  
    clf_random.fit(X_train, y_train)
    pred_proba = clf_random.predict_proba(X_test)
    
    precision, recall, _ = precision_recall_curve(y_test, pred_proba[:, 1])
    prs.append(interp(mean_recall, precision, recall))
    pr_auc = auc(recall, precision)
    aucs.append(pr_auc)
    plt.plot(recall, precision, lw=3, alpha=0.5, label='Fold %d (AUCPR = %0.2f)' % (i+1, pr_auc))

    viz = RocCurveDisplay.from_estimator(clf_random, X_test, y_test, name='ROC fold {}'.format(i+1), ax=ax, alpha=.3)

    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)

    lst_accu_stratified.append(clf_random.score(X_test, y_test))
    y_train_pred = clf_random.predict(X_train)
    train_accuracy = np.mean(y_train_pred.ravel() == y_train.ravel())*100
    train_accuracy_list.append(train_accuracy)
    y_test_pred = clf_random.predict(X_test)
    test_accuracy = np.mean(y_test_pred.ravel() == y_test.ravel())*100
    test_accuracy_list.append(test_accuracy)
    
    print('--------------------------------------------------------------------')
    print('Best Score: %s'.format(i) % clf_random.best_score_)
    print('Best Hyperparameters: %s'.format(i) % clf_random.best_params_)
    acc = accuracy_score(y_test, y_test_pred)
    prec = precision_score(y_test, y_test_pred)
    rec = recall_score(y_test, y_test_pred)
    f1 = f1_score(y_test,y_test_pred)
    accs.append(acc)
    precs.append(prec)
    recs.append(rec)
    f1_scr.append(f1)
    print(f'Accuracy: {acc}, Precision: {prec}, Recall: {rec}, f1_score: {f1}')
    i=i+1


ax.plot([0, 1], [0, 1], linestyle='--', lw=.7, color='black', label='Chance', alpha=.7)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color='blue', label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc), lw=.7, alpha=.7)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.7, label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05], title="Receiver operating characteristic for LGBM")
ax.legend(loc="lower right")
plt.show()

plt.plot([0, 1], [1, 0], linestyle='--', lw=.7, color='k', label='Luck', alpha=.7)
mean_precision = np.mean(prs, axis=0)
mean_auc = auc(mean_recall, mean_precision)
std_auc = np.std(aucs)
plt.plot(mean_precision, mean_recall, color='navy',label=r'Mean (AUCPR = %0.3f $\pm$ %0.2f)' % (mean_auc, std_auc),lw=4)
    
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('Recall' ,  fontweight = "bold" , fontsize=10)
plt.ylabel('Precision',fontweight = "bold" , fontsize=10)
plt.tick_params(axis='both', which='major', labelsize=10)
plt.legend( prop={'size':10} , loc = 0)
plt.show()

from statistics import mean
from statistics import stdev

print('List of possible accuracy: \n', lst_accu_stratified)
print('Maximum Accuracy That can be obtained from this model is: \n', max(lst_accu_stratified)*100, '%')
print('Minimum Accuracy: \n', min(lst_accu_stratified)*100, '%')
print('Overall Accuracy: \n', mean(lst_accu_stratified)*100, '%')
print('Standard Deviation is: \n', stdev(lst_accu_stratified))

print('List of possible accuracy for trainning:', train_accuracy_list)
print('List of possible accuracy for testing:', test_accuracy_list)

print(f'Mean Accuracy: {np.mean(accs)}, Mean Precision: {np.mean(precs)}, Mean Recall: {np.mean(recs)}, Mean f1_score: {np.mean(f1)}')

In [417]:
model6 = ExtraTreesClassifier(random_state=seed).fit(X1_train, y1_train)

In [418]:
clf = print_auc(model6, X1_train, y1_train, X1_test, y1_test, 'ExtraTrees Classifier')
results_df5 = pd.DataFrame(data=[['ExtraTrees Classifier', clf['f1'], clf['auc'], clf['auc_pr'], clf['recall_sc'][0], clf['recall_sc'][1], clf['precision_sc'][0], clf['precision_sc'][1]]], 
                          columns=['Model', 'f1_score', 'auc', 'auc_pr', 'recall_classe(0)', 'recall_classe(1)', 'precision_classe(0)', 'precision_classe(1)'])
results_df = results_df.append(results_df5, ignore_index=True)

results_df.style.hide_index()

Model,f1_score,auc,auc_pr,recall_classe(0),recall_classe(1),precision_classe(0),precision_classe(1)
Logistic Regression,0.888889,0.953297,0.932885,0.967033,0.875000,0.956522,0.903226
SVM,0.847458,0.966346,0.927824,0.978022,0.781250,0.927083,0.925926
Gradient Boosting,0.875000,0.959135,0.936712,0.956044,0.875000,0.956044,0.875000
Random Forest,0.852459,0.965659,0.927233,0.967033,0.812500,0.936170,0.896552
KNeibors Classifier,0.833333,0.955357,0.917124,0.967033,0.781250,0.926316,0.892857
ExtraTrees Classifier,0.852459,0.963942,0.901118,0.967033,0.812500,0.936170,0.896552


## **AdaBoost**

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

clf1 = AdaBoostClassifier(random_state=0)

cross_val = StratifiedKFold(n_splits=5)

fig, ax = plt.subplots(figsize=(5.5, 5.5))

tprs, aucs, y_real, y_prob, prs = [], [], [], [], []

mean_recall = np.linspace(0, 1, 100)
    
plt.figure(figsize=(5.5 , 5.5))

accs, precs, recs, f1_scr = [], [], [], []

mean_recall = np.linspace(0, 1, 100)
mean_fpr = np.linspace(0, 1, 100)

i=0

param_grid = {'n_estimators': randint(50, 200),
              'learning_rate': uniform(1e-5, 1),
              'base_estimator': [DecisionTreeClassifier(max_depth=1), DecisionTreeClassifier(max_depth=3), None]}

train_accuracy_list = []
test_accuracy_list = []
lst_accu_stratified = []

index_iterator = cross_val.split(X, y)

clf_random = clf_random = RandomizedSearchCV(estimator=clf1, param_distributions=param_grid, n_iter=100, scoring='f1', cv=cross_val, verbose=2, n_jobs=-1, random_state=seed)
for train_index, test_index in index_iterator:
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
  
    clf_random.fit(X_train, y_train)
    pred_proba = clf_random.predict_proba(X_test)
    
    precision, recall, _ = precision_recall_curve(y_test, pred_proba[:, 1])
    prs.append(interp(mean_recall, precision, recall))
    pr_auc = auc(recall, precision)
    aucs.append(pr_auc)
    plt.plot(recall, precision, lw=3, alpha=0.5, label='Fold %d (AUCPR = %0.2f)' % (i+1, pr_auc))

    viz = RocCurveDisplay.from_estimator(clf_random, X_test, y_test, name='ROC fold {}'.format(i+1), ax=ax, alpha=.3)

    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)

    lst_accu_stratified.append(clf_random.score(X_test, y_test))
    y_train_pred = clf_random.predict(X_train)
    train_accuracy = np.mean(y_train_pred.ravel() == y_train.ravel())*100
    train_accuracy_list.append(train_accuracy)
    y_test_pred = clf_random.predict(X_test)
    test_accuracy = np.mean(y_test_pred.ravel() == y_test.ravel())*100
    test_accuracy_list.append(test_accuracy)
    print('--------------------------------------------------------------------')
    print('Best Score: %s'.format(i) % clf_random.best_score_)
    print('Best Hyperparameters: %s'.format(i) % clf_random.best_params_)
    acc = accuracy_score(y_test, y_test_pred)
    prec = precision_score(y_test, y_test_pred)
    rec = recall_score(y_test, y_test_pred)
    f1 = f1_score(y_test,y_test_pred)
    accs.append(acc)
    precs.append(prec)
    recs.append(rec)
    f1_scr.append(f1)
    print(f'Accuracy: {acc}, Precision: {prec}, Recall: {rec}, f1_score: {f1}')
    i=i+1


ax.plot([0, 1], [0, 1], linestyle='--', lw=.7, color='black', label='Chance', alpha=.7)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color='blue', label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc), lw=.7, alpha=.7)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.7, label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05], title="Receiver operating characteristic for LGBM")
ax.legend(loc="lower right")
plt.show()

plt.plot([0, 1], [1, 0], linestyle='--', lw=.7, color='k', label='Luck', alpha=.7)
mean_precision = np.mean(prs, axis=0)
mean_auc = auc(mean_recall, mean_precision)
std_auc = np.std(aucs)
plt.plot(mean_precision, mean_recall, color='navy',label=r'Mean (AUCPR = %0.3f $\pm$ %0.2f)' % (mean_auc, std_auc),lw=4)
    
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('Recall' ,  fontweight = "bold" , fontsize=10)
plt.ylabel('Precision',fontweight = "bold" , fontsize=10)
plt.tick_params(axis='both', which='major', labelsize=10)
plt.legend( prop={'size':10} , loc = 0)
plt.show()

from statistics import mean
from statistics import stdev

print('List of possible accuracy: \n', lst_accu_stratified)
print('Maximum Accuracy That can be obtained from this model is: \n', max(lst_accu_stratified)*100, '%')
print('Minimum Accuracy: \n', min(lst_accu_stratified)*100, '%')
print('Overall Accuracy: \n', mean(lst_accu_stratified)*100, '%')
print('Standard Deviation is: \n', stdev(lst_accu_stratified))

print('List of possible accuracy for trainning:', train_accuracy_list)
print('List of possible accuracy for testing:', test_accuracy_list)

print(f'Mean Accuracy: {np.mean(accs)}, Mean Precision: {np.mean(precs)}, Mean Recall: {np.mean(recs)}, Mean f1_score: {np.mean(f1)}')

In [419]:
model7 = AdaBoostClassifier(random_state=seed).fit(X1_train, y1_train)

In [420]:
clf = print_auc(model7, X1_train, y1_train, X1_test, y1_test, 'AdaBoost Classifier')
results_df6 = pd.DataFrame(data=[['AdaBoost Classifier', clf['f1'], clf['auc'], clf['auc_pr'], clf['recall_sc'][0], clf['recall_sc'][1], clf['precision_sc'][0], clf['precision_sc'][1]]], 
                          columns=['Model', 'f1_score', 'auc', 'auc_pr', 'recall_classe(0)', 'recall_classe(1)', 'precision_classe(0)', 'precision_classe(1)'])
results_df = results_df.append(results_df6, ignore_index=True)

results_df.style.hide_index()

Model,f1_score,auc,auc_pr,recall_classe(0),recall_classe(1),precision_classe(0),precision_classe(1)
Logistic Regression,0.888889,0.953297,0.932885,0.967033,0.875000,0.956522,0.903226
SVM,0.847458,0.966346,0.927824,0.978022,0.781250,0.927083,0.925926
Gradient Boosting,0.875000,0.959135,0.936712,0.956044,0.875000,0.956044,0.875000
Random Forest,0.852459,0.965659,0.927233,0.967033,0.812500,0.936170,0.896552
KNeibors Classifier,0.833333,0.955357,0.917124,0.967033,0.781250,0.926316,0.892857
ExtraTrees Classifier,0.852459,0.963942,0.901118,0.967033,0.812500,0.936170,0.896552
AdaBoost Classifier,0.800000,0.932521,0.875056,0.923077,0.812500,0.933333,0.787879


## **Decision Tree**

from sklearn.tree import DecisionTreeClassifier

clf1 = DecisionTreeClassifier(random_state=0)

cross_val = StratifiedKFold(n_splits=5)

fig, ax = plt.subplots(figsize=(5.5, 5.5))

tprs, aucs, y_real, y_prob, prs = [], [], [], [], []

mean_recall = np.linspace(0, 1, 100)
    
plt.figure(figsize=(5.5 , 5.5))

accs, precs, recs, f1_scr = [], [], [], []

mean_recall = np.linspace(0, 1, 100)
mean_fpr = np.linspace(0, 1, 100)

i=0

param_grid = {'min_samples_split': randint(2, 20),
              'max_depth': randint(3, 10),
              'min_samples_leaf': randint(2, 10),
              'criterion' : ['gini', 'entropy', 'log_loss'],
              'max_features': ['auto', 'sqrt', 'log2']}

train_accuracy_list = []
test_accuracy_list = []
lst_accu_stratified = []

index_iterator = cross_val.split(X, y)

clf_random = clf_random = RandomizedSearchCV(estimator=clf1, param_distributions=param_grid, n_iter=100, scoring='f1', cv=cross_val, verbose=2, n_jobs=-1, random_state=seed)
for train_index, test_index in index_iterator:
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
  
    clf_random.fit(X_train, y_train)
    pred_proba = clf_random.predict_proba(X_test)
    
    precision, recall, _ = precision_recall_curve(y_test, pred_proba[:, 1])
    prs.append(interp(mean_recall, precision, recall))
    pr_auc = auc(recall, precision)
    aucs.append(pr_auc)
    plt.plot(recall, precision, lw=3, alpha=0.5, label='Fold %d (AUCPR = %0.2f)' % (i+1, pr_auc))

    viz = RocCurveDisplay.from_estimator(clf_random, X_test, y_test, name='ROC fold {}'.format(i+1), ax=ax, alpha=.3)

    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)

    lst_accu_stratified.append(clf_random.score(X_test, y_test))
    y_train_pred = clf_random.predict(X_train)
    train_accuracy = np.mean(y_train_pred.ravel() == y_train.ravel())*100
    train_accuracy_list.append(train_accuracy)
    y_test_pred = clf_random.predict(X_test)
    test_accuracy = np.mean(y_test_pred.ravel() == y_test.ravel())*100
    test_accuracy_list.append(test_accuracy)
    print('--------------------------------------------------------------------')
    print('Best Score: %s'.format(i) % clf_random.best_score_)
    print('Best Hyperparameters: %s'.format(i) % clf_random.best_params_)
    acc = accuracy_score(y_test, y_test_pred)
    prec = precision_score(y_test, y_test_pred)
    rec = recall_score(y_test, y_test_pred)
    f1 = f1_score(y_test,y_test_pred)
    accs.append(acc)
    precs.append(prec)
    recs.append(rec)
    f1_scr.append(f1)
    print(f'Accuracy: {acc}, Precision: {prec}, Recall: {rec}, f1_score: {f1}')
    i=i+1


ax.plot([0, 1], [0, 1], linestyle='--', lw=.7, color='black', label='Chance', alpha=.7)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color='blue', label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc), lw=.7, alpha=.7)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.7, label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05], title="Receiver operating characteristic for LGBM")
ax.legend(loc="lower right")
plt.show()

plt.plot([0, 1], [1, 0], linestyle='--', lw=.7, color='k', label='Luck', alpha=.7)
mean_precision = np.mean(prs, axis=0)
mean_auc = auc(mean_recall, mean_precision)
std_auc = np.std(aucs)
plt.plot(mean_precision, mean_recall, color='navy',label=r'Mean (AUCPR = %0.3f $\pm$ %0.2f)' % (mean_auc, std_auc),lw=4)
    
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('Recall' ,  fontweight = "bold" , fontsize=10)
plt.ylabel('Precision',fontweight = "bold" , fontsize=10)
plt.tick_params(axis='both', which='major', labelsize=10)
plt.legend( prop={'size':10} , loc = 0)
plt.show()

from statistics import mean
from statistics import stdev

print('List of possible accuracy: \n', lst_accu_stratified)
print('Maximum Accuracy That can be obtained from this model is: \n', max(lst_accu_stratified)*100, '%')
print('Minimum Accuracy: \n', min(lst_accu_stratified)*100, '%')
print('Overall Accuracy: \n', mean(lst_accu_stratified)*100, '%')
print('Standard Deviation is: \n', stdev(lst_accu_stratified))

print('List of possible accuracy for trainning:', train_accuracy_list)
print('List of possible accuracy for testing:', test_accuracy_list)

print(f'Mean Accuracy: {np.mean(accs)}, Mean Precision: {np.mean(precs)}, Mean Recall: {np.mean(recs)}, Mean f1_score: {np.mean(f1)}')

In [421]:
model8 = DecisionTreeClassifier(random_state=seed).fit(X1_train, y1_train)

In [422]:
clf = print_auc(model8, X1_train, y1_train, X1_test, y1_test, 'DecisionTree Classifier')
results_df7 = pd.DataFrame(data=[['DecisionTree Classifier', clf['f1'], clf['auc'], clf['auc_pr'], clf['recall_sc'][0], clf['recall_sc'][1], clf['precision_sc'][0], clf['precision_sc'][1]]], 
                          columns=['Model', 'f1_score', 'auc', 'auc_pr', 'recall_classe(0)', 'recall_classe(1)', 'precision_classe(0)', 'precision_classe(1)'])
results_df = results_df.append(results_df7, ignore_index=True)

results_df.style.hide_index()

Model,f1_score,auc,auc_pr,recall_classe(0),recall_classe(1),precision_classe(0),precision_classe(1)
Logistic Regression,0.888889,0.953297,0.932885,0.967033,0.875000,0.956522,0.903226
SVM,0.847458,0.966346,0.927824,0.978022,0.781250,0.927083,0.925926
Gradient Boosting,0.875000,0.959135,0.936712,0.956044,0.875000,0.956044,0.875000
Random Forest,0.852459,0.965659,0.927233,0.967033,0.812500,0.936170,0.896552
KNeibors Classifier,0.833333,0.955357,0.917124,0.967033,0.781250,0.926316,0.892857
ExtraTrees Classifier,0.852459,0.963942,0.901118,0.967033,0.812500,0.936170,0.896552
AdaBoost Classifier,0.800000,0.932521,0.875056,0.923077,0.812500,0.933333,0.787879
DecisionTree Classifier,0.818182,0.883413,0.839259,0.923077,0.843750,0.943820,0.794118


## **GaussianNB**


from sklearn.naive_bayes import GaussianNB

clf1 = GaussianNB()

cross_val = StratifiedKFold(n_splits=5)

fig, ax = plt.subplots(figsize=(5.5, 5.5))

tprs, aucs, y_real, y_prob, prs = [], [], [], [], []

mean_recall = np.linspace(0, 1, 100)
    
plt.figure(figsize=(5.5 , 5.5))

accs, precs, recs, f1_scr = [], [], [], []

mean_recall = np.linspace(0, 1, 100)
mean_fpr = np.linspace(0, 1, 100)

i=0

param_grid = {'var_smoothing': uniform(1e-9, 1e-6)}

train_accuracy_list = []
test_accuracy_list = []
lst_accu_stratified = []

index_iterator = cross_val.split(X, y)

clf_random = clf_random = RandomizedSearchCV(estimator=clf1, param_distributions=param_grid, n_iter=100, scoring='f1', cv=cross_val, verbose=2, n_jobs=-1, random_state=seed)
for train_index, test_index in index_iterator:
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
  
    clf_random.fit(X_train, y_train)
    pred_proba = clf_random.predict_proba(X_test)
    
    precision, recall, _ = precision_recall_curve(y_test, pred_proba[:, 1])
    prs.append(interp(mean_recall, precision, recall))
    pr_auc = auc(recall, precision)
    aucs.append(pr_auc)
    plt.plot(recall, precision, lw=3, alpha=0.5, label='Fold %d (AUCPR = %0.2f)' % (i+1, pr_auc))

    viz = RocCurveDisplay.from_estimator(clf_random, X_test, y_test, name='ROC fold {}'.format(i+1), ax=ax, alpha=.3)

    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)

    lst_accu_stratified.append(clf_random.score(X_test, y_test))
    y_train_pred = clf_random.predict(X_train)
    train_accuracy = np.mean(y_train_pred.ravel() == y_train.ravel())*100
    train_accuracy_list.append(train_accuracy)
    y_test_pred = clf_random.predict(X_test)
    test_accuracy = np.mean(y_test_pred.ravel() == y_test.ravel())*100
    test_accuracy_list.append(test_accuracy)
    print('--------------------------------------------------------------------')
    print('Best Score: %s'.format(i) % clf_random.best_score_)
    print('Best Hyperparameters: %s'.format(i) % clf_random.best_params_)
    acc = accuracy_score(y_test, y_test_pred)
    prec = precision_score(y_test, y_test_pred)
    rec = recall_score(y_test, y_test_pred)
    f1 = f1_score(y_test,y_test_pred)
    accs.append(acc)
    precs.append(prec)
    recs.append(rec)
    f1_scr.append(f1)
    print(f'Accuracy: {acc}, Precision: {prec}, Recall: {rec}, f1_score: {f1}')
    i=i+1


ax.plot([0, 1], [0, 1], linestyle='--', lw=.7, color='black', label='Chance', alpha=.7)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color='blue', label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc), lw=.7, alpha=.7)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.7, label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05], title="Receiver operating characteristic for LGBM")
ax.legend(loc="lower right")
plt.show()

plt.plot([0, 1], [1, 0], linestyle='--', lw=.7, color='k', label='Luck', alpha=.7)
mean_precision = np.mean(prs, axis=0)
mean_auc = auc(mean_recall, mean_precision)
std_auc = np.std(aucs)
plt.plot(mean_precision, mean_recall, color='navy',label=r'Mean (AUCPR = %0.3f $\pm$ %0.2f)' % (mean_auc, std_auc),lw=4)
    
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('Recall' ,  fontweight = "bold" , fontsize=10)
plt.ylabel('Precision',fontweight = "bold" , fontsize=10)
plt.tick_params(axis='both', which='major', labelsize=10)
plt.legend( prop={'size':10} , loc = 0)
plt.show()

from statistics import mean
from statistics import stdev

print('List of possible accuracy: \n', lst_accu_stratified)
print('Maximum Accuracy That can be obtained from this model is: \n', max(lst_accu_stratified)*100, '%')
print('Minimum Accuracy: \n', min(lst_accu_stratified)*100, '%')
print('Overall Accuracy: \n', mean(lst_accu_stratified)*100, '%')
print('Standard Deviation is: \n', stdev(lst_accu_stratified))

print('List of possible accuracy for trainning:', train_accuracy_list)
print('List of possible accuracy for testing:', test_accuracy_list)

print(f'Mean Accuracy: {np.mean(accs)}, Mean Precision: {np.mean(precs)}, Mean Recall: {np.mean(recs)}, Mean f1_score: {np.mean(f1)}')

model9 = GaussianNB().fit(X1_train, y1_train)

clf = print_auc(model9, X1_train, y1_train, X1_test, y1_test, 'GaussianNB')
results_df8 = pd.DataFrame(data=[['GaussianNB', clf['f1'], clf['auc'], clf['auc_pr'], clf['recall_sc'][0], clf['recall_sc'][1], clf['precision_sc'][0], clf['precision_sc'][1]]], 
                          columns=['Model', 'f1_score', 'auc', 'auc_pr', 'recall_classe(0)', 'recall_classe(1)', 'precision_classe(0)', 'precision_classe(1)'])
results_df = results_df.append(results_df8, ignore_index=True)

results_df.style.hide_index()

## **LGBM**

import lightgbm as lgb

clf1 = lgb.LGBMClassifier()

cross_val = StratifiedKFold(n_splits=5)

fig, ax = plt.subplots(figsize=(5.5, 5.5))

tprs, aucs, y_real, y_prob, prs = [], [], [], [], []

mean_recall = np.linspace(0, 1, 100)
    
plt.figure(figsize=(5.5 , 5.5))

accs, precs, recs, f1_scr = [], [], [], []

mean_recall = np.linspace(0, 1, 100)
mean_fpr = np.linspace(0, 1, 100)

i=0

param_grid = {'num_leaves': randint(10, 30),
              'max_depth': randint(3, 10),
              'learning_rate': uniform(1e-5, 1),
              'n_estimators': randint(50, 200),
              'min_child_samples': randint(10, 30),
              'subsample': uniform(0.8, 1),
              'colsample_bytree': uniform(0.8, 1),
              'class_weight' : ['balanced', None, class_weights]}

train_accuracy_list = []
test_accuracy_list = []
lst_accu_stratified = []

index_iterator = cross_val.split(X, y)

clf_random = clf_random = RandomizedSearchCV(estimator=clf1, param_distributions=param_grid, n_iter=100, scoring='f1', cv=cross_val, verbose=2, n_jobs=-1, random_state=seed)
for train_index, test_index in index_iterator:
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
  
    clf_random.fit(X_train, y_train)
    pred_proba = clf_random.predict_proba(X_test)
    
    precision, recall, _ = precision_recall_curve(y_test, pred_proba[:, 1])
    prs.append(interp(mean_recall, precision, recall))
    pr_auc = auc(recall, precision)
    aucs.append(pr_auc)
    plt.plot(recall, precision, lw=3, alpha=0.5, label='Fold %d (AUCPR = %0.2f)' % (i+1, pr_auc))

    viz = RocCurveDisplay.from_estimator(clf_random, X_test, y_test, name='ROC fold {}'.format(i+1), ax=ax, alpha=.3)

    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)

    lst_accu_stratified.append(clf_random.score(X_test, y_test))
    y_train_pred = clf_random.predict(X_train)
    train_accuracy = np.mean(y_train_pred.ravel() == y_train.ravel())*100
    train_accuracy_list.append(train_accuracy)
    y_test_pred = clf_random.predict(X_test)
    test_accuracy = np.mean(y_test_pred.ravel() == y_test.ravel())*100
    test_accuracy_list.append(test_accuracy)
    print('--------------------------------------------------------------------')
    print('Best Score: %s'.format(i) % clf_random.best_score_)
    print('Best Hyperparameters: %s'.format(i) % clf_random.best_params_)
    acc = accuracy_score(y_test, y_test_pred)
    prec = precision_score(y_test, y_test_pred)
    rec = recall_score(y_test, y_test_pred)
    f1 = f1_score(y_test,y_test_pred)
    accs.append(acc)
    precs.append(prec)
    recs.append(rec)
    f1_scr.append(f1)
    print(f'Accuracy: {acc}, Precision: {prec}, Recall: {rec}, f1_score: {f1}')
    i=i+1


ax.plot([0, 1], [0, 1], linestyle='--', lw=.7, color='black', label='Chance', alpha=.7)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color='blue', label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc), lw=.7, alpha=.7)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.7, label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05], title="Receiver operating characteristic for LGBM")
ax.legend(loc="lower right")
plt.show()

plt.plot([0, 1], [1, 0], linestyle='--', lw=.7, color='k', label='Luck', alpha=.7)
mean_precision = np.mean(prs, axis=0)
mean_auc = auc(mean_recall, mean_precision)
std_auc = np.std(aucs)
plt.plot(mean_precision, mean_recall, color='navy',label=r'Mean (AUCPR = %0.3f $\pm$ %0.2f)' % (mean_auc, std_auc),lw=4)
    
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('Recall' ,  fontweight = "bold" , fontsize=10)
plt.ylabel('Precision',fontweight = "bold" , fontsize=10)
plt.tick_params(axis='both', which='major', labelsize=10)
plt.legend( prop={'size':10} , loc = 0)
plt.show()

from statistics import mean
from statistics import stdev

print('List of possible accuracy: \n', lst_accu_stratified)
print('Maximum Accuracy That can be obtained from this model is: \n', max(lst_accu_stratified)*100, '%')
print('Minimum Accuracy: \n', min(lst_accu_stratified)*100, '%')
print('Overall Accuracy: \n', mean(lst_accu_stratified)*100, '%')
print('Standard Deviation is: \n', stdev(lst_accu_stratified))

print('List of possible accuracy for trainning:', train_accuracy_list)
print('List of possible accuracy for testing:', test_accuracy_list)

print(f'Mean Accuracy: {np.mean(accs)}, Mean Precision: {np.mean(precs)}, Mean Recall: {np.mean(recs)}, Mean f1_score: {np.mean(f1)}')

In [423]:
model10 = lgb.LGBMClassifier(random_state=seed).fit(X1_train, y1_train)

In [424]:
clf = print_auc(model10, X1_train, y1_train, X1_test, y1_test, 'LGBN')
results_df9 = pd.DataFrame(data=[['LGBM', clf['f1'], clf['auc'], clf['auc_pr'], clf['recall_sc'][0], clf['recall_sc'][1], clf['precision_sc'][0], clf['precision_sc'][1]]], 
                          columns=['Model', 'f1_score', 'auc', 'auc_pr', 'recall_classe(0)', 'recall_classe(1)', 'precision_classe(0)', 'precision_classe(1)'])
results_df = results_df.append(results_df9, ignore_index=True)

results_df.style.hide_index()

Model,f1_score,auc,auc_pr,recall_classe(0),recall_classe(1),precision_classe(0),precision_classe(1)
Logistic Regression,0.888889,0.953297,0.932885,0.967033,0.875000,0.956522,0.903226
SVM,0.847458,0.966346,0.927824,0.978022,0.781250,0.927083,0.925926
Gradient Boosting,0.875000,0.959135,0.936712,0.956044,0.875000,0.956044,0.875000
Random Forest,0.852459,0.965659,0.927233,0.967033,0.812500,0.936170,0.896552
KNeibors Classifier,0.833333,0.955357,0.917124,0.967033,0.781250,0.926316,0.892857
ExtraTrees Classifier,0.852459,0.963942,0.901118,0.967033,0.812500,0.936170,0.896552
AdaBoost Classifier,0.800000,0.932521,0.875056,0.923077,0.812500,0.933333,0.787879
DecisionTree Classifier,0.818182,0.883413,0.839259,0.923077,0.843750,0.943820,0.794118
LGBM,0.830769,0.958791,0.933096,0.934066,0.843750,0.944444,0.818182


## **XGB**

import xgboost as xgb

clf1 = xgb.XGBClassifier(objective='binary:logistic', scale_pos_weight=imbalance_ratio)

cross_val = StratifiedKFold(n_splits=5)

fig, ax = plt.subplots(figsize=(5.5, 5.5))

tprs, aucs, y_real, y_prob, prs = [], [], [], [], []

mean_recall = np.linspace(0, 1, 100)
    
plt.figure(figsize=(5.5 , 5.5))


accs, precs, recs, f1_scr = [], [], [], []

mean_recall = np.linspace(0, 1, 100)
mean_fpr = np.linspace(0, 1, 100)

i=0

param_grid = {'learning_rate' : uniform(1e-5, 1),
              'max_depth' : randint(3, 25),
              'gamma' : uniform(0.1, 1),
              'subsample': uniform(0.5, 1),
              'colsample_bytree': uniform(0.5, 1),
              'scale_pos_weight': [1, imbalance_ratio]}

train_accuracy_list = []
test_accuracy_list = []
lst_accu_stratified = []

index_iterator = cross_val.split(X, y)

clf_random = clf_random = RandomizedSearchCV(estimator=clf1, param_distributions=param_grid, n_iter=100, scoring='f1', cv=cross_val, verbose=2, n_jobs=-1, random_state=seed)
for train_index, test_index in index_iterator:
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
  
    clf_random.fit(X_train, y_train)
    pred_proba = clf_random.predict_proba(X_test)
    
    precision, recall, _ = precision_recall_curve(y_test, pred_proba[:, 1])
    prs.append(interp(mean_recall, precision, recall))
    pr_auc = auc(recall, precision)
    aucs.append(pr_auc)
    plt.plot(recall, precision, lw=3, alpha=0.5, label='Fold %d (AUCPR = %0.2f)' % (i+1, pr_auc))

    viz = RocCurveDisplay.from_estimator(clf_random, X_test, y_test, name='ROC fold {}'.format(i+1), ax=ax, alpha=.3)

    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)

    lst_accu_stratified.append(clf_random.score(X_test, y_test))
    y_train_pred = clf_random.predict(X_train)
    train_accuracy = np.mean(y_train_pred.ravel() == y_train.ravel())*100
    train_accuracy_list.append(train_accuracy)
    y_test_pred = clf_random.predict(X_test)
    test_accuracy = np.mean(y_test_pred.ravel() == y_test.ravel())*100
    test_accuracy_list.append(test_accuracy)
    print('--------------------------------------------------------------------')
    print('Best Score: %s'.format(i) % clf_random.best_score_)
    print('Best Hyperparameters: %s'.format(i) % clf_random.best_params_)
    acc = accuracy_score(y_test, y_test_pred)
    prec = precision_score(y_test, y_test_pred)
    rec = recall_score(y_test, y_test_pred)
    f1 = f1_score(y_test,y_test_pred)
    accs.append(acc)
    precs.append(prec)
    recs.append(rec)
    f1_scr.append(f1)
    print(f'Accuracy: {acc}, Precision: {prec}, Recall: {rec}, f1_score: {f1}')
    i=i+1


ax.plot([0, 1], [0, 1], linestyle='--', lw=.7, color='black', label='Chance', alpha=.7)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color='blue', label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc), lw=.7, alpha=.7)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.7, label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05], title="Receiver operating characteristic for LGBM")
ax.legend(loc="lower right")
plt.show()

plt.plot([0, 1], [1, 0], linestyle='--', lw=.7, color='k', label='Luck', alpha=.7)
mean_precision = np.mean(prs, axis=0)
mean_auc = auc(mean_recall, mean_precision)
std_auc = np.std(aucs)
plt.plot(mean_precision, mean_recall, color='navy',label=r'Mean (AUCPR = %0.3f $\pm$ %0.2f)' % (mean_auc, std_auc),lw=4)
    
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('Recall' ,  fontweight = "bold" , fontsize=10)
plt.ylabel('Precision',fontweight = "bold" , fontsize=10)
plt.tick_params(axis='both', which='major', labelsize=10)
plt.legend( prop={'size':10} , loc = 0)
plt.show()

from statistics import mean
from statistics import stdev

print('List of possible accuracy: \n', lst_accu_stratified)
print('Maximum Accuracy That can be obtained from this model is: \n', max(lst_accu_stratified)*100, '%')
print('Minimum Accuracy: \n', min(lst_accu_stratified)*100, '%')
print('Overall Accuracy: \n', mean(lst_accu_stratified)*100, '%')
print('Standard Deviation is: \n', stdev(lst_accu_stratified))

print('List of possible accuracy for trainning:', train_accuracy_list)
print('List of possible accuracy for testing:', test_accuracy_list)

print(f'Mean Accuracy: {np.mean(accs)}, Mean Precision: {np.mean(precs)}, Mean Recall: {np.mean(recs)}, Mean f1_score: {np.mean(f1)}')

In [425]:
model11 = xgb.XGBClassifier(objective='binary:logistic', random_state=seed).fit(X1_train, y1_train)

In [426]:
clf = print_auc(model11, X1_train, y1_train, X1_test, y1_test, 'XGB Classifer')
results_df10 = pd.DataFrame(data=[['XGB Classifer', clf['f1'], clf['auc'], clf['auc_pr'], clf['recall_sc'][0], clf['recall_sc'][1], clf['precision_sc'][0], clf['precision_sc'][1]]], 
                          columns=['Model', 'f1_score', 'auc', 'auc_pr', 'recall_classe(0)', 'recall_classe(1)', 'precision_classe(0)', 'precision_classe(1)'])
results_df = results_df.append(results_df10, ignore_index=True)

results_df.style.hide_index()

Model,f1_score,auc,auc_pr,recall_classe(0),recall_classe(1),precision_classe(0),precision_classe(1)
Logistic Regression,0.888889,0.953297,0.932885,0.967033,0.875000,0.956522,0.903226
SVM,0.847458,0.966346,0.927824,0.978022,0.781250,0.927083,0.925926
Gradient Boosting,0.875000,0.959135,0.936712,0.956044,0.875000,0.956044,0.875000
Random Forest,0.852459,0.965659,0.927233,0.967033,0.812500,0.936170,0.896552
KNeibors Classifier,0.833333,0.955357,0.917124,0.967033,0.781250,0.926316,0.892857
ExtraTrees Classifier,0.852459,0.963942,0.901118,0.967033,0.812500,0.936170,0.896552
AdaBoost Classifier,0.800000,0.932521,0.875056,0.923077,0.812500,0.933333,0.787879
DecisionTree Classifier,0.818182,0.883413,0.839259,0.923077,0.843750,0.943820,0.794118
LGBM,0.830769,0.958791,0.933096,0.934066,0.843750,0.944444,0.818182
XGB Classifer,0.857143,0.959135,0.926096,0.956044,0.843750,0.945652,0.870968


In [427]:
results_df.to_csv('results_dfROS.csv', index=False)